Instalarea pachetelor necesare

In [ ]:
!pip uninstall -y torch torchvision torchaudio timm numpy
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2+cu117 --index-url https://download.pytorch.org/whl/cu117
!pip install numpy==1.24.4 timm==0.9.2
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu117/torch2.0/index.html

from mmcv.ops import DeformConv2d
print("mmcv-full funcționează!")

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: timm 1.0.15
Uninstalling timm-1.0.15:
  Successfully uninstalled timm-1.0.15
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Looking in indexes: https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/1.8 GB 155.8 MB/s eta 0:00:08

Descărcare proiect

In [ ]:
import os

# Navigare în directorul content
os.chdir('/content')

REPO_URL = "https://github.com/Fabian12Florin/DCHF-Bloc-de-fuziune-pentru-estimarea-adancimii-din-imagini.git"
REPO_NAME = "/DCHF-Bloc-de-fuziune-pentru-estimarea-adancimii-din-imagini"

!git clone {REPO_URL}

os.chdir(f'/content/{REPO_NAME}')
print(f"Directorul curent: {os.getcwd()}")

In [ ]:
from google.colab import files
import os

print("Încarcă modelul:")
uploaded = files.upload()

# Locația modelului
model_dir = "train_log/DCHF/model"
os.makedirs(model_dir, exist_ok=True)

# Se mută modelul în directorul DCHF
for filename in uploaded.keys():
    if filename.endswith('.pth'):
        model_path = f"{model_dir}/{filename}"
        os.rename(filename, model_path)
        file_size = os.path.getsize(model_path) / (1024*1024)
        print(f"Modelul a fost pus la: {model_path} ({file_size:.1f} MB)")

print("Model încărcat!")


In [ ]:
import os
from IPython.display import Image, display

# Directorul pentru output
output_dir = "/content/depth_predictions"
os.makedirs(output_dir, exist_ok=True)

os.chdir('/content/DCHF-Bloc-de-fuziune-pentru-estimarea-adancimii-din-imagini')

# Verifică directorul pentru imagini
if os.path.exists("example_images"):
    images = [f for f in os.listdir("example_images") if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"S-au găsit {len(images)} imagini:")
    for img in images:
        print(f"  - {img}")

    # Arată prima imagine
    if images:
        img_path = f"example_images/{images[0]}"
        display(Image(img_path, width=400))
        print(f"Preview: {images[0]}")
else:
    print("Directorul example_images nu a fost găsit.")

In [ ]:
import time

# Configurare
image_path = "example_images"
config_path = "options/TiO-Depth/eval/tio_depth-swint-m_384_kitti.yaml"
model_path = "/content/DCHF-Bloc-de-fuziune-pentru-estimarea-adancimii-din-imagini/train_log/DCHF/model/last_model.pth"
output_dir = "/content/depth_predictions"

print("Au început predicțiile!")
print(f"Imagini: {image_path}")
print(f"Config: {config_path}")
print(f"Model: {model_path}")
print(f"Output: {output_dir}")

# Verifică prezența tuturor directoarelor
for name, path in [("Config", config_path), ("Model", model_path), ("Imagini", image_path)]:
    if os.path.exists(path):
        print(f"✅ {name}: A fost găsit")
    else:
        print(f"❌ {name}: Nu a fost găsit - {path}")

print("\n" + "="*60)
start_time = time.time()

# Rulează predicțiile
cmd = f"python predict.py --image_path {image_path} --exp_opts {config_path} --model_path {model_path} --out_dir {output_dir}"
print(f"Executing: {cmd}")

result = os.system(cmd)
duration = time.time() - start_time

if result == 0:
    print(f"\n SUCCESS! A durat {duration:.1f} secunde")
else:
    print(f"\n⚠️ S-a terminat cu warning în {duration:.1f} secunde")

# Verifică predicțiile
if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    print(f"📁 S-au generat {len(files)} fișiere:")
    for f in sorted(files):
        size = os.path.getsize(f"{output_dir}/{f}") / 1024
        print(f"  - {f} ({size:.1f} KB)")

    if len(files) > 0:
        print("\n✅ Estimare corectă!")
    else:
        print("\n❌ Nu au fost generate fișiere")
else:
    print("❌ Directorul pentru output nu a fost găsit!")


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

output_dir = "/content/depth_predictions"
input_dir = "example_images"

print("Vizualizare rezultate")

if os.path.exists(output_dir) and os.listdir(output_dir):
    # Obține fișierele
    png_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])

    for i, png_file in enumerate(png_files):
        base_name = png_file.replace('_depth.png', '').replace('_pred.png', '')

        # Verifică dacă există imaginile
        input_file = None
        for ext in ['.png', '.jpg', '.jpeg']:
            candidate = os.path.join(input_dir, base_name + ext)
            if os.path.exists(candidate):
                input_file = candidate
                break

        # Dimensiuni figură
        fig, axes = plt.subplots(1, 1, figsize=(12, 5))

        # Imagini output
        depth_img = Image.open(os.path.join(output_dir, png_file))
        axes.imshow(depth_img)
        axes.set_title(f'Depth: {png_file}')
        axes.axis('off')

        plt.tight_layout()
        plt.show()

else:
    print("❌ Nu există rezultate")
    print(f"📁 Verifică directorul {output_dir}")

In [ ]:
!python /content/Licenta---Estimarea-adancimii-din-imagini/predict.py --image_path /content/Licenta---Estimarea-adancimii-din-imagini/example_images --exp_opts /content/Licenta---Estimarea-adancimii-din-imagini/options/TiO-Depth/eval/tio_depth-swint-m_384_kitti.yaml --model_path /content/train_log/2025-06-16_12h32m02s_TiO-Depth-MFM0-ACVDAV21/model/best_model-refdepth.pth --out_dir /content/depth_predictions